In [35]:
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
import os
import csv
import tensorflow as tf
from functools import partial
from scipy.signal import savgol_filter
from tensorflow.keras.layers import InputLayer
from time import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import f1_score, roc_auc_score
import matplotlib.pyplot as plt
from prettytable import PrettyTable as pt
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from tensorflow import keras
from sklearn.model_selection import KFold
from keras import layers
import argparse
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from keras import regularizers
from keras.layers import Input, Dropout, Activation, BatchNormalization, Add
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from keras.layers import  GlobalAveragePooling1D,AveragePooling1D ,MaxPool1D,MaxPooling2D,ZeroPadding1D, LSTM, Bidirectional
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Flatten, Conv2D, MaxPool2D
from numpy import unique
#from colorama import Fore, Back, Style
from prettytable import PrettyTable
filterwarnings('ignore')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# train_dataset = pd.read_csv('/content/drive/MyDrive/new_physionet.csv')
# test_dataset = pd.read_csv('/content/drive/MyDrive/new_test.csv')

In [3]:
train_dataset = pd.read_csv('C:/Users/tousi.KCRND/Downloads/MAML/new_physionet.csv')
test_dataset = pd.read_csv('C:/Users/tousi.KCRND/Downloads/MAML/new_test.csv')

In [4]:
train_dataset['label'] = train_dataset['label'].replace([2,3],0)

In [5]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_data = train_dataset[:train_size]
val_data = train_dataset[train_size:]

X_train = train_dataset.drop(columns=['label']).values
y_train = train_dataset['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values


In [6]:
label_1 = test_dataset[test_dataset['label'] == 1].sample(n=50)
label_0 = test_dataset[test_dataset['label'] == 0].sample(n=50)
selected_rows_1 = pd.concat([label_1, label_0])
Meta_df_2 = shuffle(selected_rows_1).reset_index(drop=True)

In [7]:
Meta_df_1 = test_dataset.drop(index=selected_rows_1.index)
Meta_df_1 = shuffle(Meta_df_1).reset_index(drop=True)

In [8]:
X_train_meta = Meta_df_1.drop(columns=['label']).values
y_train_meta = Meta_df_1['label'].values
X_test_meta = Meta_df_2.drop(columns=['label']).values
y_test_meta = Meta_df_2['label'].values

In [ ]:
# X_train =  train_dataset.drop(columns=['label'])
# y_train =  train_dataset['label']

# X_test =  test_dataset.drop(columns=['label'])
# y_test =  test_dataset['label']

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_train_meta = scaler.transform(X_train_meta)
X_test_meta = scaler.transform(X_test_meta)

In [10]:
"""Convolutional layers used by MAML model."""
## NOTE: You do not need to modify this block but you will need to use it.
seed = 123
import tensorflow as tf

def conv_block_1d(inp, cweight, bweight, bn, activation=tf.nn.relu):
    """ Perform conv, batch norm, and nonlinearity """
    conv_output = tf.nn.conv1d(input=inp, filters=cweight, stride=1, padding='SAME') + bweight
    normed = bn(conv_output)
    normed = activation(normed)
    return normed

class ConvLayers(tf.keras.layers.Layer):
    def __init__(self, channels, dim_hidden, dim_output, signal_length):
        super(ConvLayers, self).__init__()
        self.channels = channels
        self.dim_hidden = dim_hidden
        self.dim_output = dim_output
        self.signal_length = signal_length

        weights = {}

        dtype = tf.float32
        weight_initializer = tf.keras.initializers.GlorotUniform()
        k = 3

        weights['conv1'] = tf.Variable(weight_initializer(shape=[k, self.channels, self.dim_hidden]), name='conv1', dtype=dtype)
        weights['b1'] = tf.Variable(tf.zeros([self.dim_hidden]), name='b1')
        self.bn1 = tf.keras.layers.BatchNormalization(name='bn1')
        weights['conv2'] = tf.Variable(weight_initializer(shape=[k, self.dim_hidden, self.dim_hidden]), name='conv2', dtype=dtype)
        weights['b2'] = tf.Variable(tf.zeros([self.dim_hidden]), name='b2')
        self.bn2 = tf.keras.layers.BatchNormalization(name='bn2')
        weights['conv3'] = tf.Variable(weight_initializer(shape=[k, self.dim_hidden, self.dim_hidden]), name='conv3', dtype=dtype)
        weights['b3'] = tf.Variable(tf.zeros([self.dim_hidden]), name='b3')
        self.bn3 = tf.keras.layers.BatchNormalization(name='bn3')
        weights['conv4'] = tf.Variable(weight_initializer(shape=[k, self.dim_hidden, self.dim_hidden]), name='conv4', dtype=dtype)
        weights['b4'] = tf.Variable(tf.zeros([self.dim_hidden]), name='b4')
        self.bn4 = tf.keras.layers.BatchNormalization(name='bn4')
        weights['w5'] = tf.Variable(weight_initializer(shape=[self.dim_hidden, self.dim_output]), name='w5', dtype=dtype)
        weights['b5'] = tf.Variable(tf.zeros([self.dim_output]), name='b5')
        self.conv_weights = weights

    def call(self, inp, weights):
        channels = self.channels
        inp = tf.reshape(inp, [-1, self.signal_length, channels])
        hidden1 = conv_block_1d(inp, weights['conv1'], weights['b1'], self.bn1)
        hidden2 = conv_block_1d(hidden1, weights['conv2'], weights['b2'], self.bn2)
        hidden3 = conv_block_1d(hidden2, weights['conv3'], weights['b3'], self.bn3)
        hidden4 = conv_block_1d(hidden3, weights['conv4'], weights['b4'], self.bn4)
        hidden4 = tf.reduce_mean(hidden4, axis=1)
        return tf.matmul(hidden4, weights['w5']) + weights['b5']

# Example usage
# channels = 1
# dim_hidden = 64
# dim_output = 4
# signal_length = 1000

# model = ConvLayers(channels, dim_hidden, dim_output, signal_length)


In [76]:
# mobilenet =  MobileNetV2((1000,1,1), 4, 1.0)
# trainable_variables = mobilenet.get_trainable_variables()
# trainable_weights = mobilenet.get_trainable_weights()

# print("Trainable variables:", trainable_variables)
# print("Trainable weights:", trainable_weights)


In [ ]:
# def build_meta_model(input_shape):
#     #inner_model = build_model(input_shape)
#     inner_model =  MobileNetv2((X_train.shape[1],1), 1, 1.0)
#     input_layer = layers.Input(shape=input_shape)
#     inner_weights = inner_model(input_layer)
#     meta_output = layers.Dense(1)(inner_weights)

#     model = keras.Model(inputs=input_layer, outputs=meta_output)

#     return model

# DataGenerator

In [11]:
class DataGenerator(object):
    def __init__(self, X_train, y_train, num_classes, num_samples_per_class,
                 X_val=None, y_val=None, X_train_meta=None, y_train_meta=None,
                 X_test_meta=None, y_test_meta=None, num_meta_test_classes=None,
                 num_meta_test_samples_per_class=None):
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        
        self.X_train_meta = X_train_meta
        self.y_train_meta = y_train_meta
        self.X_test_meta = X_test_meta
        self.y_test_meta = y_test_meta
        
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes
        self.num_meta_test_samples_per_class = num_meta_test_samples_per_class
        self.num_meta_test_classes = num_meta_test_classes
        self.dim_input = X_train.shape[1] 
        self.dim_output = num_classes

    def sample_batch(self, batch_type, batch_size, shuffle=True):
        if batch_type == "meta_train":
            X, y = self.X_train, self.y_train
        elif batch_type == "meta_val":
            X, y = self.X_val, self.y_val
            
        elif batch_type == "meta_fine_tune":
            X, y = self.X_train_meta, self.y_train_meta
        else:  # "meta_test"
            X = self.X_test_meta
            y = self.y_test_meta
            num_classes = self.num_meta_test_classes
            num_samples_per_class = self.num_meta_test_samples_per_class


        all_signal_batches = []
        all_label_batches = []
        for _ in range(batch_size):
            sampled_signals = []
            sampled_labels = []
            sampled_classes = np.random.choice(np.unique(y), self.num_classes, replace=True)
            for class_label in sampled_classes:
                class_indices = np.where(y == class_label)[0]
                num_samples_available = len(class_indices)
                num_samples_to_select = min(num_samples_available, self.num_samples_per_class)
                selected_indices = np.random.choice(class_indices, num_samples_to_select, replace=False)
                if isinstance(selected_indices, int):
                    selected_indices = [selected_indices]  # Convert single integer to list
                sampled_signals.extend(X[selected_indices])
                sampled_labels.extend([class_label] * num_samples_to_select)  # Use num_samples_to_select instead of self.num_samples_per_class

            # Convert to numpy arrays
            sampled_signals = np.array(sampled_signals)
            sampled_labels = np.array(sampled_labels)

            # Shuffle within the batch if needed
            if shuffle:
                indices = np.arange(sampled_signals.shape[0])
                np.random.shuffle(indices)
                sampled_signals = sampled_signals[indices]
                sampled_labels = sampled_labels[indices]

            # One-hot encode labels
            labels_one_hot = np.eye(self.num_classes)[sampled_labels]

            all_signal_batches.append(sampled_signals)
            all_label_batches.append(labels_one_hot)

        all_signal_batches = np.stack(all_signal_batches)
        all_label_batches = np.stack(all_label_batches)

        return all_signal_batches, all_label_batches


In [34]:
# num_classes = 2
# num_samples_per_class = 5
# num_meta_test_classes = 2
# num_meta_test_samples_per_class = 5

# # Assuming you have defined X_train, y_train, X_val, y_val, X_train_meta, y_train_meta, X_test_meta, and y_test_meta
# data_generator = DataGenerator(X_train, y_train, num_classes, num_samples_per_class,
#                                X_val, y_val, X_train_meta, y_train_meta, X_test_meta,
#                                y_test_meta, num_meta_test_classes, num_meta_test_samples_per_class)

# batch_type = "meta_test"
# batch_size = 25

# # Generate a batch for meta-testing
# signals_batch, labels_batch = data_generator.sample_batch(batch_type, batch_size)


# MAML

In [12]:
# def cross_entropy_loss(pred, label, k_shot):
#     pred_float = tf.cast(pred, tf.float32)
#     label_float = tf.cast(label, tf.float32)
#     return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred_float, labels=label_float)/k_shot)


def cross_entropy_loss(pred, label, k_shot):
    if not isinstance(pred, tf.Tensor):
        raise TypeError("pred must be a tf.Tensor")
    if not isinstance(label, tf.Tensor):
        raise TypeError("label must be a tf.Tensor")
    
    # Check compatibility of shapes (optional)
    if pred.shape != label.shape:
        raise ValueError("Shape mismatch between pred and label")

    pred_float = tf.cast(pred, tf.float32)
    label_float = tf.cast(label, tf.float32)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred_float, labels=label_float) / k_shot)

def accuracy(labels, predictions):
    if len(predictions.shape) > 1:
        predictions = tf.argmax(predictions, axis=1)
    if len(labels.shape) > 1:
        labels = tf.argmax(labels, axis=1)

    labels = tf.cast(labels, dtype=tf.float32)  # Cast labels to float32
    predictions = tf.cast(predictions, dtype=tf.float32)

    correct_predictions = tf.equal(labels, predictions)
    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

# def accuracy(labels, predictions):
#   return tf.reduce_mean(tf.cast(tf.equal(labels, predictions), dtype=tf.float32))


In [129]:


class MAML(tf.keras.Model):
    def __init__(self, dim_input=1000, dim_output=2,
                 num_inner_updates=1,
                 inner_update_lr=0.4, k_shot=5, learn_inner_update_lr=False):
        super(MAML, self).__init__()
        self.dim_input = dim_input
        self.dim_output = dim_output
        self.inner_update_lr = inner_update_lr
        self.num_inner_updates = num_inner_updates
        self.loss_func = partial(cross_entropy_loss, k_shot=k_shot)
        self.dim_hidden = 32
        self.channels = 1
        #self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        self.k_shot = k_shot
        #self.feature_extractor =  MobileNetV2((1000,1,1), 4, 1.0)
        self.conv_layers = ConvLayers(self.channels, self.dim_hidden, self.dim_output, self.dim_input)
        self.learn_inner_update_lr = learn_inner_update_lr
        if self.learn_inner_update_lr:
          self.inner_update_lr_dict = {}
          for key in self.conv_layers.conv_weights.keys():
            self.inner_update_lr_dict[key] = [tf.Variable(self.inner_update_lr, name='inner_update_lr_%s_%d' % (key, j)) for j in range(num_inner_updates)]

    @tf.function
    def call(self, inp, meta_batch_size=20, num_inner_updates=1):

      def task_inner_loop(inp, reuse=True, meta_batch_size=25, num_inner_updates=1):
        input_tr, input_ts, label_tr, label_ts = inp
        task_output_tr_pre, task_loss_tr_pre, task_accuracy_tr_pre = None, None, None
        task_outputs_ts, task_losses_ts, task_accuracies_ts = [], [], []

        #weights_optimized = self.feature_extractor.get_weights()
        #weights_optimized = self.feature_extractor.model.get_weights()
        #trainable_variables =self.feature_extractor.model.get_trainable_variables()
        weights = self.conv_layers.conv_weights
        #print('weights:',weights)
        # weights_optimized = {}
        # for layer in self.feature_extractor.model.layers:
        #     for weight in layer.weights:
        #          weights_optimized[weight.name] = weight

        #weights_optimized = {weight: weight for weight in self.feature_extractor.model.weights}
        weights_optimized = dict(zip(weights.keys(), [weights[key] for key in weights.keys()]))

        with tf.GradientTape(persistent=True) as inner_tape:
            for j in range(num_inner_updates):
              input_tr_expanded = tf.expand_dims(input_tr, axis=0)
              #logits_tr = self.feature_extractor.model(input_tr_expanded)
              logits_tr = self.conv_layers(input_tr_expanded, weights_optimized)
              label_tr = tf.reshape(label_tr, (1, -1)) 
              # print("logits_tr:", logits_tr.shape)
              # print("label_tr:", label_tr.shape)


              loss_tr_j = self.loss_func(logits_tr, label_tr)
              #print('loss_tr_j',loss_tr_j)

              if (task_output_tr_pre is None) and (task_loss_tr_pre is None) and (task_accuracy_tr_pre is None):
                    task_output_tr_pre = logits_tr
                    task_loss_tr_pre = loss_tr_j

                    task_accuracy_tr_pre = accuracy(
                        label_tr,
                        tf.nn.softmax(task_output_tr_pre)
                    )

            #grads = inner_tape.gradient(loss_tr_j, weights_optimized.values())
              grads = inner_tape.gradient(loss_tr_j, list(weights_optimized.values()))
              #print('grads',grads)
              grad_dict = dict(zip(weights_optimized.keys(), grads))
            
              
              if self.learn_inner_update_lr:
                  # update optimized weights using learnable learning rate
                  weights_optimized = dict(zip(weights_optimized.keys(),
                                            [weights_optimized[key] - self.inner_update_lr_dict[key][j] * grad_dict[key] for key in weights_optimized.keys()]))
                  
              else:
                  # update optimized weights
                  weights_optimized = dict(zip(weights_optimized.keys(),
                                      [weights_optimized[key] - self.inner_update_lr * grad_dict[key] for key in weights_optimized.keys()]))

            # compute logits for test data
            #print('done')
              input_ts_expanded = tf.expand_dims(input_ts, axis=0)
              #logits_ts = self.feature_extractor.model(input_ts_expanded)
              logits_ts = self.conv_layers(input_ts_expanded, weights_optimized)

            # compute loss
              #label_ts_expanded = tf.squeeze(label_ts, axis=0)
              label_ts = tf.reshape(label_ts, (1, -1)) 
              loss_ts_j = self.loss_func(label_ts, logits_ts)
            # add to the task output list
              task_outputs_ts.append(logits_ts)
            # compute task loss on test data
              task_losses_ts.append(loss_ts_j)

      # compute accuracies after inner update step
        for j in range(num_inner_updates):
          predictions_ts = tf.nn.softmax(task_outputs_ts[j], axis=1)
          task_accuracy_ts = accuracy(label_ts, predictions_ts)

          task_accuracies_ts.append(task_accuracy_ts)

        # print("task_output_tr_pre:", task_output_tr_pre)
        # print("task_loss_tr_pre:", task_loss_tr_pre)
        # print("task_accuracy_tr_pre:", task_accuracy_tr_pre)
        task_output = [task_output_tr_pre, task_outputs_ts, task_loss_tr_pre, task_losses_ts, task_accuracy_tr_pre,
                      task_accuracies_ts]
        return task_output


      # def task_inner_loop_partial(inp_sample, meta_batch_size=25, num_inner_updates=1):
      #     return task_inner_loop(
      #         (inp_sample[0], inp_sample[1], inp_sample[2], inp_sample[3]),
      #         False,
      #         meta_batch_size,
      #         num_inner_updates
      #     )

      input_tr, input_ts, label_tr, label_ts = inp

      dummy_result = task_inner_loop((tf.identity(input_tr[0]),
                                        tf.identity(input_ts[0]),
                                        tf.identity(label_tr[0]),
                                        tf.identity(label_ts[0])),
                              False,
                              meta_batch_size,
                              num_inner_updates)
      
      
    #   output_signature = (tf.float32, [tf.float32] * num_inner_updates, tf.float32, [tf.float32] * num_inner_updates,
    #                 tf.float32, [tf.float32] * num_inner_updates)
      out_dtype = [tf.float32, [tf.float32]*num_inner_updates, tf.float32, [tf.float32]*num_inner_updates]
      out_dtype.extend([tf.float32, [tf.float32]*num_inner_updates])
      task_inner_loop_partial = partial(task_inner_loop, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)
      result = tf.map_fn(task_inner_loop_partial,
                      elems=(input_tr, input_ts, label_tr, label_ts),
                      dtype=out_dtype,
                      parallel_iterations=meta_batch_size)
      return result
      
#     result = tf.map_fn(task_inner_loop_partial,
#                 elems=(input_tr, input_ts, label_tr, label_ts),
#                 fn_output_signature=output_signature,
#                 parallel_iterations=meta_batch_size)
# return result


# outer loop

In [130]:
import pickle
import random


def outer_train_step(inp, model, optim, meta_batch_size=25, num_inner_updates=1):
  with tf.GradientTape(persistent=False) as outer_tape:
    result = model(inp, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    outputs_tr, outputs_ts, losses_tr_pre, losses_ts, accuracies_tr_pre, accuracies_ts = result

    total_losses_ts = [tf.reduce_mean(loss_ts) for loss_ts in losses_ts]

  gradients = outer_tape.gradient(total_losses_ts[-1], model.trainable_variables)
 # print('gradients',gradients)
  optim.apply_gradients(zip(gradients, model.trainable_variables))

  total_loss_tr_pre = tf.reduce_mean(losses_tr_pre)
  total_accuracy_tr_pre = tf.reduce_mean(accuracies_tr_pre)
  total_accuracies_ts = [tf.reduce_mean(accuracy_ts) for accuracy_ts in accuracies_ts]

  return outputs_tr, outputs_ts, total_loss_tr_pre, total_losses_ts, total_accuracy_tr_pre, total_accuracies_ts

def outer_eval_step(inp, model, meta_batch_size=25, num_inner_updates=1):
  result = model(inp, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

  outputs_tr, outputs_ts, losses_tr_pre, losses_ts, accuracies_tr_pre, accuracies_ts = result

  total_loss_tr_pre = tf.reduce_mean(losses_tr_pre)
  total_losses_ts = [tf.reduce_mean(loss_ts) for loss_ts in losses_ts]

  total_accuracy_tr_pre = tf.reduce_mean(accuracies_tr_pre)
  total_accuracies_ts = [tf.reduce_mean(accuracy_ts) for accuracy_ts in accuracies_ts]

  return outputs_tr, outputs_ts, total_loss_tr_pre, total_losses_ts, total_accuracy_tr_pre, total_accuracies_ts


def meta_train_fn(model, data_generator,
               n_way=2, meta_train_iterations=200, meta_batch_size=25,
               k_shot=5, num_inner_updates=1, meta_lr=0.001):

  SUMMARY_INTERVAL = 10
  SAVE_INTERVAL = 100
  PRINT_INTERVAL = SUMMARY_INTERVAL
  TEST_PRINT_INTERVAL = SUMMARY_INTERVAL
  LOG_CSV_INTERVAL = SUMMARY_INTERVAL

  path = 'C:/Users/tousi.KCRND/Downloads/MAML/weights/'

  pre_accuracies, post_accuracies = [], []

  num_classes = 2
  optimizer = tf.keras.optimizers.Adam(learning_rate=meta_lr)

  task_train_acc_pre_optim, task_train_acc_post_optim, task_test_acc_pre_optim, task_test_acc_post_optim = 0., 0., 0., 0.

  for itr in range(meta_train_iterations):

    input_, label_ = data_generator.sample_batch(batch_type='meta_train', batch_size=meta_batch_size, shuffle=True)
    #print('input shape:',input_.shape)

    input_tr = np.reshape(input_[:, :n_way*k_shot, :], newshape=(-1, np.prod(data_generator.dim_input)))
    input_ts = np.reshape(input_[:, n_way*k_shot:, :], newshape=(-1, np.prod(data_generator.dim_input)))
    label_tr = np.reshape(label_[:, :n_way*k_shot, :], newshape=(-1, data_generator.dim_output))
    label_ts = np.reshape(label_[:, n_way*k_shot:, :], newshape=(-1, data_generator.dim_output))

    input_tr = np.reshape(input_tr, newshape=(-1, 1000, 1, 1))  
    input_ts = np.reshape(input_ts, newshape=(-1, 1000, 1, 1))

    # print('input_tr:',input_tr.shape)
    # print('input_ts:',input_ts.shape)
    # print('label tr:', label_tr.shape)
    # print('label_ts:',label_ts.shape)



    inp = (input_tr, input_ts, label_tr, label_ts)

    result = outer_train_step(inp, model, optimizer, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    if itr % SUMMARY_INTERVAL == 0:
      pre_accuracies.append(result[-2])
      post_accuracies.append(result[-1][-1])

    if (itr!=0) and itr % PRINT_INTERVAL == 0:
      task_train_acc_pre_optim = np.mean(pre_accuracies)
      task_train_acc_post_optim = np.mean(post_accuracies)
      print_str = 'Iteration %d: pre-inner-loop train accuracy: %.5f, post-inner-loop test accuracy: %.5f' % (itr, np.mean(pre_accuracies), np.mean(post_accuracies))
      print(print_str)
      pre_accuracies, post_accuracies = [], []

    if (itr!=0) and itr % TEST_PRINT_INTERVAL == 0:

      input_, label_ = data_generator.sample_batch(batch_type='meta_val', batch_size=meta_batch_size, shuffle=True)
      input_tr = np.reshape(input_[:, :n_way*k_shot, :], newshape=(-1, np.prod(data_generator.dim_input)))
      input_ts = np.reshape(input_[:, n_way*k_shot:, :], newshape=(-1, np.prod(data_generator.dim_input)))
      label_tr = np.reshape(label_[:, :n_way*k_shot, :], newshape=(-1, data_generator.dim_output))
      label_ts = np.reshape(label_[:, n_way*k_shot:, :], newshape=(-1, data_generator.dim_output))
      input_tr = np.reshape(input_tr, newshape=(-1, 1000, 1, 1))  
      input_ts = np.reshape(input_ts, newshape=(-1, 1000, 1, 1))


      inp = (input_tr, input_ts, label_tr, label_ts)
      result = outer_eval_step(inp, model, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

      task_test_acc_pre_optim = result[-2].numpy()
      task_test_acc_post_optim = result[-1][-1].numpy()
      print('Meta-validation pre-inner-loop train accuracy: %.5f, meta-validation post-inner-loop test accuracy: %.5f' % (result[-2], result[-1][-1]))

    if (itr != 0) and itr % SAVE_INTERVAL == 0:
        model_file = os.path.join(path, f'{itr}_model.h5')
        print("Saving to ", model_file)
        model.save_weights(model_file)


NUM_META_TEST_POINTS = 100

def meta_test_fn(model, data_generator, n_way=2, meta_batch_size=25, k_shot=5,
              num_inner_updates=1):

  #num_classes = data_generator.num_classes
  path = 'C:/Users/tousi.KCRND/Downloads/MAML/weights/'

  num_classes = 2

  np.random.seed(1)
  random.seed(1)

  meta_test_accuracies = []

  for _ in range(NUM_META_TEST_POINTS):

    input_, label_ = data_generator.sample_batch(batch_type='meta_test', batch_size=meta_batch_size, shuffle=True)
    input_tr = np.reshape(input_[:, :n_way*k_shot, :], newshape=(-1, np.prod(data_generator.dim_input)))
    input_ts = np.reshape(input_[:, n_way*k_shot:, :], newshape=(-1, np.prod(data_generator.dim_input)))
    label_tr = np.reshape(label_[:, :n_way*k_shot, :], newshape=(-1, data_generator.dim_output))
    label_ts = np.reshape(label_[:, n_way*k_shot:, :], newshape=(-1, data_generator.dim_output))

    input_tr = np.reshape(input_tr, newshape=(-1, 1000, 1, 1))  
    input_ts = np.reshape(input_ts, newshape=(-1, 1000, 1, 1))


    inp = (input_tr, input_ts, label_tr, label_ts)
    result = outer_eval_step(inp, model, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    meta_test_accuracies.append(result[-1][-1])

  meta_test_accuracies = np.array(meta_test_accuracies)
  means = np.mean(meta_test_accuracies)
  stds = np.std(meta_test_accuracies)
  ci95 = 1.96*stds/np.sqrt(NUM_META_TEST_POINTS)

  print('Mean meta-test accuracy/loss, stddev, and confidence intervals')
  print((means, stds, ci95))


def run_maml(n_way, k_shot, meta_batch_size=25, meta_lr=0.001,
             inner_update_lr=0.4, num_inner_updates=1,
             learn_inner_update_lr=False,
             resume=False, resume_itr=0,
             meta_train=True,
             meta_train_iterations=1000, meta_train_k_shot=-1,
             meta_train_inner_update_lr=-1):

  # call data_generator and get data with k_shot*2 samples per class

  path = 'C:/Users/tousi.KCRND/Downloads/MAML/weights/'
  data_generator = DataGenerator(X_train, y_train, n_way, k_shot*2,
                               X_val, y_val, X_train_meta, y_train_meta, X_test_meta,
                               y_test_meta, n_way, k_shot*2)




  # set up MAML model
  dim_output = data_generator.dim_output
  dim_input = data_generator.dim_input
  #print('1')
  model = MAML(dim_input,
              dim_output,
              num_inner_updates=num_inner_updates,
              inner_update_lr=inner_update_lr,
              k_shot=k_shot,
              learn_inner_update_lr=learn_inner_update_lr)
  #print('2')
  if meta_train_k_shot == -1:
    meta_train_k_shot = k_shot
  if meta_train_inner_update_lr == -1:
    meta_train_inner_update_lr = inner_update_lr

  # exp_string = 'cls_'+str(n_way)+'.mbs_'+str(meta_batch_size) + '.k_shot_' + str(meta_train_k_shot) + '.inner_numstep_' + str(num_inner_updates) + '.inner_updatelr_' + str(meta_train_inner_update_lr) + '.learn_inner_update_lr_' + str(learn_inner_update_lr)
  # logfile = exp_string+'.csv'

  if meta_train:
    meta_train_fn(model, data_generator,
                  n_way, meta_train_iterations, meta_batch_size, k_shot,
                  num_inner_updates, meta_lr)
  else:
    meta_batch_size = 1
    modeldir = 'C:/Users/tousi.KCRND/Downloads/MAML/weights'
    exp_string = '900_model.h5'
    model_file = os.path.join(modeldir, exp_string)

    if os.path.exists(model_file):
        # Load weights
        # model.load_weights(model_file)
        # print("Model weights loaded successfully from", model_file)
        meta_test_fn(model, data_generator, n_way, meta_batch_size, k_shot, num_inner_updates)
    else:
      print("Model file not found at", model_file)


    #model_file = tf.train.latest_checkpoint(modeldir + '/' + exp_string)
    # print("Restoring model weights from ", model_file)
    # model.load_weights(model_file)

  # path = 'C:/Users/tousi.KCRND/Downloads/MAML/weights/'
  # latest_checkpoint = tf.train.latest_checkpoint(path)
  # if latest_checkpoint:
  #     print("Restoring model weights from ", latest_checkpoint)
  #     model.load_weights(latest_checkpoint)
  # else:
  #     print("No checkpoint found in ", path)



# RUN

In [18]:
run_maml(n_way=2, k_shot=5, inner_update_lr=0.1, num_inner_updates=1)

Instructions for updating:
Use fn_output_signature instead
Iteration 10: pre-inner-loop train accuracy: 0.58800, post-inner-loop test accuracy: 0.81600
Meta-validation pre-inner-loop train accuracy: 0.48000, meta-validation post-inner-loop test accuracy: 0.68000
Iteration 20: pre-inner-loop train accuracy: 0.54400, post-inner-loop test accuracy: 0.73600
Meta-validation pre-inner-loop train accuracy: 0.47600, meta-validation post-inner-loop test accuracy: 0.74400
Iteration 30: pre-inner-loop train accuracy: 0.57200, post-inner-loop test accuracy: 0.76800
Meta-validation pre-inner-loop train accuracy: 0.42400, meta-validation post-inner-loop test accuracy: 0.80000
Iteration 40: pre-inner-loop train accuracy: 0.40400, post-inner-loop test accuracy: 0.77600
Meta-validation pre-inner-loop train accuracy: 0.51200, meta-validation post-inner-loop test accuracy: 0.66400
Iteration 50: pre-inner-loop train accuracy: 0.49200, post-inner-loop test accuracy: 0.78400
Meta-validation pre-inner-loop t

In [21]:
# test the model
run_maml(n_way=2, k_shot=5, inner_update_lr=0.01, num_inner_updates=1, meta_train=False)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.7353334, 0.29294747, 0.023440679640714147)


In [48]:
run_maml(n_way=2, k_shot=10, inner_update_lr=0.01, num_inner_updates=1)

Iteration 10: pre-inner-loop train accuracy: 0.46100, post-inner-loop test accuracy: 0.74000
Meta-validation pre-inner-loop train accuracy: 0.49200, meta-validation post-inner-loop test accuracy: 0.76800
Iteration 20: pre-inner-loop train accuracy: 0.57600, post-inner-loop test accuracy: 0.81200
Meta-validation pre-inner-loop train accuracy: 0.45400, meta-validation post-inner-loop test accuracy: 0.70800
Iteration 30: pre-inner-loop train accuracy: 0.51000, post-inner-loop test accuracy: 0.70400
Meta-validation pre-inner-loop train accuracy: 0.47200, meta-validation post-inner-loop test accuracy: 0.76000
Iteration 40: pre-inner-loop train accuracy: 0.54200, post-inner-loop test accuracy: 0.69600
Meta-validation pre-inner-loop train accuracy: 0.52200, meta-validation post-inner-loop test accuracy: 0.77200
Iteration 50: pre-inner-loop train accuracy: 0.51000, post-inner-loop test accuracy: 0.62400
Meta-validation pre-inner-loop train accuracy: 0.52000, meta-validation post-inner-loop tes

In [49]:
run_maml(n_way=2, k_shot=10, inner_update_lr=0.4, num_inner_updates=1, meta_train=False)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.74333334, 0.26798114, 0.021442957231667194)


In [18]:
# Run MAML with learnable learning rate
run_maml(n_way=2, k_shot=5, inner_update_lr=4.0, num_inner_updates=1, learn_inner_update_lr=True)

Instructions for updating:
Use fn_output_signature instead
Iteration 10: pre-inner-loop train accuracy: 0.55000, post-inner-loop test accuracy: 0.75600
Meta-validation pre-inner-loop train accuracy: 0.38800, meta-validation post-inner-loop test accuracy: 0.73600
Iteration 20: pre-inner-loop train accuracy: 0.50400, post-inner-loop test accuracy: 0.77600
Meta-validation pre-inner-loop train accuracy: 0.52800, meta-validation post-inner-loop test accuracy: 0.78400
Iteration 30: pre-inner-loop train accuracy: 0.41200, post-inner-loop test accuracy: 0.80000
Meta-validation pre-inner-loop train accuracy: 0.65200, meta-validation post-inner-loop test accuracy: 0.75200
Iteration 40: pre-inner-loop train accuracy: 0.54800, post-inner-loop test accuracy: 0.73600
Meta-validation pre-inner-loop train accuracy: 0.52800, meta-validation post-inner-loop test accuracy: 0.67200
Iteration 50: pre-inner-loop train accuracy: 0.59600, post-inner-loop test accuracy: 0.68800
Meta-validation pre-inner-loop t

In [19]:
run_maml(n_way=2, k_shot=5, inner_update_lr=0.4, num_inner_updates=1, meta_train=False, learn_inner_update_lr=True)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.76000005, 0.2842534, 0.05571366417407989)


In [20]:
run_maml(n_way=2, k_shot=7, inner_update_lr=0.4, num_inner_updates=1)

Iteration 10: pre-inner-loop train accuracy: 0.38571, post-inner-loop test accuracy: 0.74857
Meta-validation pre-inner-loop train accuracy: 0.41714, meta-validation post-inner-loop test accuracy: 0.77143
Iteration 20: pre-inner-loop train accuracy: 0.47714, post-inner-loop test accuracy: 0.68000
Meta-validation pre-inner-loop train accuracy: 0.39143, meta-validation post-inner-loop test accuracy: 0.66857
Iteration 30: pre-inner-loop train accuracy: 0.45143, post-inner-loop test accuracy: 0.74857
Meta-validation pre-inner-loop train accuracy: 0.52286, meta-validation post-inner-loop test accuracy: 0.76571
Iteration 40: pre-inner-loop train accuracy: 0.45429, post-inner-loop test accuracy: 0.70286
Meta-validation pre-inner-loop train accuracy: 0.52000, meta-validation post-inner-loop test accuracy: 0.78286
Iteration 50: pre-inner-loop train accuracy: 0.56571, post-inner-loop test accuracy: 0.73143
Meta-validation pre-inner-loop train accuracy: 0.54000, meta-validation post-inner-loop tes

In [22]:
run_maml(n_way=2, k_shot=10, inner_update_lr=0.4, num_inner_updates=1, meta_train=False)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.77700007, 0.26677895, 0.05228867340087891)


Finetune

In [131]:
def meta_fine_tune(model, data_generator,
               n_way=2, meta_train_iterations=200, meta_batch_size=25,
               k_shot=5, num_inner_updates=1, meta_lr=0.0001):

  SUMMARY_INTERVAL = 10
  SAVE_INTERVAL = 100
  PRINT_INTERVAL = SUMMARY_INTERVAL
  TEST_PRINT_INTERVAL = SUMMARY_INTERVAL
  LOG_CSV_INTERVAL = SUMMARY_INTERVAL

  path = 'C:/Users/tousi.KCRND/Downloads/MAML/finetune/'

  pre_accuracies, post_accuracies = [], []

  num_classes = 2
  optimizer = tf.keras.optimizers.Adam(learning_rate=meta_lr)

  task_train_acc_pre_optim, task_train_acc_post_optim, task_test_acc_pre_optim, task_test_acc_post_optim = 0., 0., 0., 0.

  for itr in range(meta_train_iterations):

    input_, label_ = data_generator.sample_batch(batch_type='meta_fine_tune', batch_size=meta_batch_size, shuffle=True)
    #print('input shape:',input_.shape)

    input_tr = np.reshape(input_[:, :n_way*k_shot, :], newshape=(-1, np.prod(data_generator.dim_input)))
    input_ts = np.reshape(input_[:, n_way*k_shot:, :], newshape=(-1, np.prod(data_generator.dim_input)))
    label_tr = np.reshape(label_[:, :n_way*k_shot, :], newshape=(-1, data_generator.dim_output))
    label_ts = np.reshape(label_[:, n_way*k_shot:, :], newshape=(-1, data_generator.dim_output))

    input_tr = np.reshape(input_tr, newshape=(-1, 1000, 1, 1))  
    input_ts = np.reshape(input_ts, newshape=(-1, 1000, 1, 1))


    inp = (input_tr, input_ts, label_tr, label_ts)

    result = outer_train_step(inp, model, optimizer, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    if itr % SUMMARY_INTERVAL == 0:
      pre_accuracies.append(result[-2])
      post_accuracies.append(result[-1][-1])

    if (itr!=0) and itr % PRINT_INTERVAL == 0:
      task_train_acc_pre_optim = np.mean(pre_accuracies)
      task_train_acc_post_optim = np.mean(post_accuracies)
      print_str = 'Iteration %d: pre-inner-loop train accuracy: %.5f, post-inner-loop test accuracy: %.5f' % (itr, np.mean(pre_accuracies), np.mean(post_accuracies))
      print(print_str)
      pre_accuracies, post_accuracies = [], []


    if (itr != 0) and itr % SAVE_INTERVAL == 0:
        model_file = os.path.join(path, f'{itr}_model_.h5')
        print("Saving to ", model_file)
        model.save_weights(model_file)


In [132]:
def run_finetune(n_way, k_shot, meta_batch_size=25, meta_lr=0.0001,
             inner_update_lr=0.4, num_inner_updates=1,
             learn_inner_update_lr=False,
             resume=False, resume_itr=0,
             meta_train=True,
             meta_train_iterations=200, meta_train_k_shot=-1,
             meta_train_inner_update_lr=-1):

  # call data_generator and get data with k_shot*2 samples per class

  path = 'C:/Users/tousi.KCRND/Downloads/MAML/finetune/'

  data_generator = DataGenerator(X_train, y_train, n_way, k_shot*2,
                               X_val, y_val, X_train_meta, y_train_meta, X_test_meta,
                               y_test_meta, n_way, k_shot*2)


  # set up MAML model
  dim_output = data_generator.dim_output
  dim_input = data_generator.dim_input
  #print('1')
  model = MAML(dim_input,
              dim_output,
              num_inner_updates=num_inner_updates,
              inner_update_lr=inner_update_lr,
              k_shot=k_shot,
              learn_inner_update_lr=learn_inner_update_lr)
  #print('2')
  if meta_train_k_shot == -1:
    meta_train_k_shot = k_shot
  if meta_train_inner_update_lr == -1:
    meta_train_inner_update_lr = inner_update_lr


  if meta_train:
    meta_fine_tune(model, data_generator,
                  n_way, meta_train_iterations, meta_batch_size, k_shot,
                  num_inner_updates, meta_lr)
  else:
    meta_batch_size = 1
    # latest_checkpoint = tf.train.latest_checkpoint(path)
    # print("Restoring model weights from ", latest_checkpoint)
    # model.load_weights(latest_checkpoint)
    meta_test_fn(model, data_generator, n_way, meta_batch_size, k_shot, num_inner_updates)

In [28]:
run_finetune(n_way=2, k_shot=5, inner_update_lr=0.4, num_inner_updates=1)

Iteration 10: pre-inner-loop train accuracy: 0.52200, post-inner-loop test accuracy: 0.73600
Iteration 20: pre-inner-loop train accuracy: 0.45200, post-inner-loop test accuracy: 0.75200
Iteration 30: pre-inner-loop train accuracy: 0.54400, post-inner-loop test accuracy: 0.75200
Iteration 40: pre-inner-loop train accuracy: 0.45200, post-inner-loop test accuracy: 0.62400
Iteration 50: pre-inner-loop train accuracy: 0.43200, post-inner-loop test accuracy: 0.76800
Iteration 60: pre-inner-loop train accuracy: 0.54000, post-inner-loop test accuracy: 0.84000
Iteration 70: pre-inner-loop train accuracy: 0.47600, post-inner-loop test accuracy: 0.76000
Iteration 80: pre-inner-loop train accuracy: 0.54000, post-inner-loop test accuracy: 0.65600
Iteration 90: pre-inner-loop train accuracy: 0.51200, post-inner-loop test accuracy: 0.73600
Iteration 100: pre-inner-loop train accuracy: 0.55200, post-inner-loop test accuracy: 0.69600
Saving to  C:/Users/tousi.KCRND/Downloads/MAML/finetune/100_model_.h5

In [29]:
run_finetune(n_way=2, k_shot=5, inner_update_lr=0.4, num_inner_updates=1, meta_train=False)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.76000005, 0.2842534, 0.05571366417407989)


In [32]:
run_finetune(n_way=2, k_shot=10, inner_update_lr=0.4, num_inner_updates=1, meta_train=False)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.77700007, 0.26677895, 0.05228867340087891)


In [133]:
run_finetune(n_way=2, k_shot=15, inner_update_lr=0.4, num_inner_updates=1, meta_train=False)

Mean meta-test accuracy/loss, stddev, and confidence intervals
(0.7513333, 0.25679302, 0.05033143234252929)
